In [26]:
from util import helpers
import wikipediaapi
import spacy
import re
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import opennre
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from operator import itemgetter

In [10]:
# !pip install wikipedia-api spacy transformers sentence_transformers pytest

# !pip3 uninstall torch torchvision torchaudio #--extra-index-url https://download.pytorch.org/whl/cu113

# !python -m spacy download en_core_web_lg

2022-04-15 14:18:58,684 - root - INFO - Initializing word embedding with word2vec.


In [27]:

entity = [{'type': 'uri', 'uri': 'http://yago-knowledge.org/resource/Borussia_Dortmund'}][0]['uri'].rsplit('/', 1)[-1]
noHints = [[{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '56903', 'uri': '56903', 'prefix_entity': "'56903'"}, {'type': 'uri', 'value': 'countryOfOrigin', 'uri': 'http://schema.org/countryOfOrigin', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:countryOfOrigin'}, {'type': 'uri', 'value': 'United_States', 'uri': 'http://yago-knowledge.org/resource/United_States', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:United\\_States'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '21762', 'uri': '21762', 'prefix_entity': "'21762'"}, {'type': 'uri', 'value': 'countryOfOrigin', 'uri': 'http://schema.org/countryOfOrigin', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:countryOfOrigin'}, {'type': 'uri', 'value': 'India', 'uri': 'http://yago-knowledge.org/resource/India', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:India'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '11454', 'uri': '11454', 'prefix_entity': "'11454'"}, {'type': 'uri', 'value': 'countryOfOrigin', 'uri': 'http://schema.org/countryOfOrigin', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:countryOfOrigin'}, {'type': 'uri', 'value': 'United_Kingdom', 'uri': 'http://yago-knowledge.org/resource/United_Kingdom', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:United\\_Kingdom'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '8714', 'uri': '8714', 'prefix_entity': "'8714'"}, {'type': 'uri', 'value': 'countryOfOrigin', 'uri': 'http://schema.org/countryOfOrigin', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:countryOfOrigin'}, {'type': 'uri', 'value': 'France', 'uri': 
'http://yago-knowledge.org/resource/France', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:France'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '7149', 'uri': '7149', 'prefix_entity': "'7149'"}, {'type': 'uri', 'value': 'countryOfOrigin', 'uri': 'http://schema.org/countryOfOrigin', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:countryOfOrigin'}, {'type': 'uri', 'value': 'Germany', 'uri': 'http://yago-knowledge.org/resource/Germany', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:Germany'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '4030', 'uri': '4030', 'prefix_entity': "'4030'"}, {'type': 'uri', 'value': 'countryOfOrigin', 'uri': 'http://schema.org/countryOfOrigin', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:countryOfOrigin'}, {'type': 'uri', 'value': 'Canada', 'uri': 'http://yago-knowledge.org/resource/Canada', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:Canada'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '3855', 'uri': '3855', 'prefix_entity': "'3855'"}, {'type': 'uri', 'value': 'countryOfOrigin', 'uri': 'http://schema.org/countryOfOrigin', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:countryOfOrigin'}, {'type': 'uri', 'value': 'Japan', 'uri': 'http://yago-knowledge.org/resource/Japan', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:Japan'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '3473', 'uri': '3473', 'prefix_entity': "'3473'"}, {'type': 'uri', 'value': 'type', 'uri': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 'prefix': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#', 'prefix_entity': 'x2:type'}, {'type': 'uri', 'value': 'Movie', 'uri': 'http://schema.org/Movie', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:Movie'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '2768', 'uri': '2768', 'prefix_entity': "'2768'"}, {'type': 'uri', 'value': 'countryOfOrigin', 'uri': 'http://schema.org/countryOfOrigin', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:countryOfOrigin'}, {'type': 'uri', 'value': 'Australia', 'uri': 'http://yago-knowledge.org/resource/Australia', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:Australia'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 
'literal', 'value': '1787', 'uri': '1787', 'prefix_entity': "'1787'"}, {'type': 'uri', 'value': 'countryOfOrigin', 'uri': 'http://schema.org/countryOfOrigin', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:countryOfOrigin'}, {'type': 'uri', 'value': 'Soviet_Union', 'uri': 'http://yago-knowledge.org/resource/Soviet_Union', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:Soviet\\_Union'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '183', 'uri': '183', 'prefix_entity': "'183'"}, {'type': 'uri', 'value': 'nationality', 'uri': 'http://schema.org/nationality', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:nationality'}, {'type': 'uri', 'value': 'India', 'uri': 'http://yago-knowledge.org/resource/India', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:India'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '155', 'uri': '155', 'prefix_entity': "'155'"}, {'type': 'uri', 'value': 'nationality', 'uri': 'http://schema.org/nationality', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:nationality'}, {'type': 'uri', 'value': 'Canada', 'uri': 'http://yago-knowledge.org/resource/Canada', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:Canada'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '36', 'uri': '36', 'prefix_entity': "'36'"}, {'type': 'uri', 'value': 'hasOccupation', 'uri': 'http://schema.org/hasOccupation', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:hasOccupation'}, {'type': 'uri', 'value': 'Film_producer', 'uri': 'http://yago-knowledge.org/resource/Film_producer', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:Film\\_producer'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '21', 'uri': '21', 'prefix_entity': "'21'"}, {'type': 'uri', 'value': 'hasOccupation', 'uri': 'http://schema.org/hasOccupation', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:hasOccupation'}, {'type': 'uri', 'value': 'Screenwriter', 'uri': 'http://yago-knowledge.org/resource/Screenwriter', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:Screenwriter'}], [{'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '20', 'uri': '20', 'prefix_entity': "'20'"}, {'type': 'uri', 'value': 'hasOccupation', 'uri': 'http://schema.org/hasOccupation', 'prefix': 'http://schema.org/', 'prefix_entity': 'x0:hasOccupation'}, {'type': 'uri', 'value': 'Film_director', 'uri': 'http://yago-knowledge.org/resource/Film_director', 'prefix': 'http://yago-knowledge.org/resource/', 'prefix_entity': 'x1:Film\\_director'}]]

falseRelations = [[helpers.parseTriple(noHints[i])[1],helpers.parseTriple(noHints[i])[2]] for i in range(len(noHints))]

wiki_wiki = wikipediaapi.Wikipedia(
        language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI
)

p_wiki = wiki_wiki.page('{0}'.format(entity))

classifier = pipeline("fill-mask")#,  model = "xlm-roberta-base")

model_embedding = SentenceTransformer('all-mpnet-base-v2')
model_re = opennre.get_model('wiki80_cnn_softmax')

nlp = spacy.load('en_core_web_lg')

doc = nlp(p_wiki.text)

No model was supplied, defaulted to distilroberta-base (https://huggingface.co/distilroberta-base)
2022-04-15 13:32:42,476 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-mpnet-base-v2
2022-04-15 13:32:52,247 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device: cuda
2022-04-15 13:32:54,869 - wikipediaapi - INFO - Request URL: https://en.wikipedia.org/w/api.php?action=query&prop=extracts&titles=Borussia_Dortmund&explaintext=1&exsectionformat=wiki


In [141]:
s = 'The Los Angeles Lakers is a good team. They are an American professional basketball team based in Los Angeles.'
# s = 'The United States of America (U.S.A. or USA), commonly known as the United States (U.S. or US) or simply America, is a country primarily located in North America.'
subject = ['Los Angeles Lakers', 'Los Angeles', 'Lakers', 'Dortmund'] # TODO: use PoS on the subject.
# subject = ['United States', 'States'] # TODO: use PoS on the subject.
falseO = 'American'
# falseO = 'North America'

In [69]:
def cosine_sim(x, y):
    return cosine_similarity(model_embedding.encode(x).reshape(1, -1), model_embedding.encode(y).reshape(1, -1))

In [114]:
def extract_relations(sentence, subj, obj, reverse=False): 
    relations = []
    for sub_subj in subj:
        if sub_subj in sentence:
            head_s = sentence.index(sub_subj)
            head_e = head_s + len(sub_subj)
            tail_s = sentence.index(obj)
            tail_e = tail_s + len(obj)
            
            result = model_re.infer({'text': sentence, 'h': {'pos': (head_s, head_e)}, 't': {'pos': (tail_s, tail_e)}})
            if reverse:
                result = model_re.infer({'text': sentence, 't': {'pos': (head_s, head_e)}, 'h': {'pos': (tail_s, tail_e)}})
                
            relations.append((sub_subj, result, obj))
    return relations

In [97]:
def validate_relations(relations, falsePO):
    best = (0, relations[0])
    for relation in relations:
        sim = cosine_sim(f'{relation[1][0]} {relation[2]}', falsePO)
        print(relation)
        if best[0] < sim:
            best = (sim, relation)
    return best

In [140]:
s

'The Los Angeles Lakers is a good team. They are an American professional basketball team based in Los Angeles.'

In [142]:
relations = extract_relations(s, subject, falseO)
print(relations)
validate_relations(relations, "country of origin United States")

[('Los Angeles Lakers', ('sport', 0.5515372157096863), 'American'), ('Los Angeles', ('sport', 0.5515372157096863), 'American'), ('Lakers', ('country of citizenship', 0.8018515706062317), 'American')]


Batches: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 88.32it/s]


('Los Angeles Lakers', ('sport', 0.5515372157096863), 'American')


Batches: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 84.93it/s]


('Los Angeles', ('sport', 0.5515372157096863), 'American')


Batches: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 97.32it/s]

('Lakers', ('country of citizenship', 0.8018515706062317), 'American')


(array([[0.586882]], dtype=float32),
 ('Lakers', ('country of citizenship', 0.8018515706062317), 'American'))

In [7]:
subject = ['Borussia Dortmund', 'Borussia', 'Dortmund']
falsePO = "country of origin Germany"
falseO = "Germany" 

candidates = []

# TODO: pick a more robust sentence splitter.
for sentence in doc.sents:
    for token in sentence:
        chosen = False
        if (token.text == "Borussia" or token.text == "Dortmund") and token.dep_ == "nsubj":
            # use coreference relations; whether Swift is actually taylor or whether her is referring to taylor as well!
            # candidates.append(sentence)
            chosen = True
            
        if chosen:
            similar_objs = [[cosine_sim(token.text, falseObject)[0], token] for token in sentence]
            similar_objs.sort(key=itemgetter(0), reverse=True)
            best_obj = similar_objs[0]    
        
            candidates.append([sentence, best_obj[0], best_obj[1]])

            
            
# for index, candidate in enumerate(candidates):
#     similar_objs = [[cosine_sim(token.text, falseObject)[0], token] for token in candidate]
    
#     similar_objs.sort(key=itemgetter(0), reverse=True)
#     best_obj = similar_objs[0]    
    
#     candidates[index] = [candidate, best_obj[0], best_obj[1]]

candidates.sort(key=itemgetter(1), reverse=True)
print(candidates[:5])

[[Ballspielverein Borussia 09 e. V. Dortmund, commonly known as Borussia Dortmund (German pronunciation: [boˈʁʊsi̯aː ˈdɔɐ̯tmʊnt] (listen)), BVB (pronounced [beːfaʊ̯ˈbeː] (listen)), or simply Dortmund (pronounced [ˈdɔʁtmʊnt] (listen)), is a German professional sports club based in Dortmund, North Rhine-Westphalia., array([0.6959425], dtype=float32), German], [
Founded in 1909 by eighteen football players from Dortmund, the football team is part of a large membership-based sports club with more than 145,000 members, making Borussia Dortmund the second largest sports club by membership in Germany., array([1.0000002], dtype=float32), Germany], [Since 1974, Dortmund have played their home games at Westfalenstadion; the stadium is the largest in Germany, and Dortmund has the highest average attendance of any association football club in the world., array([1.0000002], dtype=float32), Germany], [Since 1974, Dortmund have played their home games at Westfalenstadion; the stadium is the largest i

In [ ]:
for c in candidates[:5]:
    for sub_subject in subject:
        head_ 
        tail_start = c[0].index(c[2])
        tail_end = tail_start + len(c[2])
    
    

In [17]:
candidates.sort(key=itemgetter(1), reverse=True)
for i in candidates[:10]:
    print(i, '\n')

[In terms of Deloitte's annual Football Money League, Dortmund was in 2015 ranked as the second richest sports club in Germany, and the 12th richest football team in the world., array([1.0000002], dtype=float32), Germany] 

[Since 1974, Dortmund have played their home games at Westfalenstadion; the stadium is the largest in Germany, and Dortmund has the highest average attendance of any association football club in the world., array([1.0000002], dtype=float32), Germany] 

[Since 1974, Dortmund have played their home games at Westfalenstadion; the stadium is the largest in Germany, and Dortmund has the highest average attendance of any association football club in the world., array([1.0000002], dtype=float32), Germany] 

[
Founded in 1909 by eighteen football players from Dortmund, the football team is part of a large membership-based sports club with more than 145,000 members, making Borussia Dortmund the second largest sports club by membership in Germany., array([1.0000002], dtype=fl

In [91]:
classifier(f"{s}. United States <mask> Los Angelos Lakers ")

[{'score': 0.3337801396846771,
  'token': 6772,
  'token_str': ' Lakers',
  'sequence': 'The Los Angeles Lakers are an American professional basketball team based in Los Angeles.. United States Lakers Los Angelos Lakers '},
 {'score': 0.09170623868703842,
  'token': 12610,
  'token_str': ' Basketball',
  'sequence': 'The Los Angeles Lakers are an American professional basketball team based in Los Angeles.. United States Basketball Los Angelos Lakers '},
 {'score': 0.03219366818666458,
  'token': 12190,
  'token_str': ' Clippers',
  'sequence': 'The Los Angeles Lakers are an American professional basketball team based in Los Angeles.. United States Clippers Los Angelos Lakers '},
 {'score': 0.029032375663518906,
  'token': 2762,
  'token_str': ' NBA',
  'sequence': 'The Los Angeles Lakers are an American professional basketball team based in Los Angeles.. United States NBA Los Angelos Lakers '},
 {'score': 0.024331482127308846,
  'token': 10433,
  'token_str': ' Soccer',
  'sequence': '

In [33]:

falseRelation = "Borussia Dortmund country of origin Germany"

candidates = []

# TODO: pick a more robust sentence splitter.
for sentence in doc.sents:
    for token in sentence:
        if (token.text == "Borussia" or token.text == "Dortmund") and token.dep_ == "nsubj":
            # use coreference relations; whether Swift is actually taylor or whether her is referring to taylor as well!
            candidates.append([cosine_sim(sentence.text, falseRelation), sentence])

candidates.sort(key=itemgetter(0), reverse=True)
print(candidates[:5])

[[array([[0.77792245]], dtype=float32), Ballspielverein Borussia 09 e. V. Dortmund, commonly known as Borussia Dortmund (German pronunciation: [boˈʁʊsi̯aː ˈdɔɐ̯tmʊnt] (listen)), BVB (pronounced [beːfaʊ̯ˈbeː] (listen)), or simply Dortmund (pronounced [ˈdɔʁtmʊnt] (listen)), is a German professional sports club based in Dortmund, North Rhine-Westphalia.], [array([[0.7135918]], dtype=float32), 
Founded in 1909 by eighteen football players from Dortmund, the football team is part of a large membership-based sports club with more than 145,000 members, making Borussia Dortmund the second largest sports club by membership in Germany.], [array([[0.69223356]], dtype=float32), Borussia Dortmund are one of four German clubs to win the Bundesliga and DFB-Pokal double, along with Bayern Munich, 1.], [array([[0.67901766]], dtype=float32), Borussia Dortmund were the second German club to win the Intercontinental Cup, after Bayern Munich in 1976.As defending champions Dortmund reached the Champions Lea

In [35]:
update = False
finalSentence = None
for candidate in candidates[:5]:
    # objects = [print(token, token.dep_) for token in candidate[1]]
    objects = [token for token in candidate[1] if token.dep_ == 'dobj' or token.dep_ == 'pobj']
    similarities = [(cosine_sim(o.text, 'Germany'), o) for o in objects]
    similarities.sort(reverse=True)
    print('All objects: ', objects, 'from', candidate)
    print('sim', similarities)
    obj = similarities[0]
    print('\n selected object', obj, '\n')
    if classifier(f"{candidate[1].text} Borussia Dortmund <mask> {obj[1].text}.")[0]["score"] >= .008: # TODO: consider a higher threshhold. WHat about ambiguity?
        result = classifier(f"{candidate[1].text} Borussia Dortmund <mask> {obj[1]}.")[0]
        pred = result['token_str']
        print('result ', result)
        if cosine_sim(f'{pred} {obj[1].text}', 'country of origin Germany') > 0.6: # TODO: change threshold!!
            update = True
            finalSentence = result['sequence']
            break
            
print(finalSentence, update)

All objects:  [Dortmund, ˈdɔʁtmʊnt, Dortmund] from [array([[0.77792245]], dtype=float32), Ballspielverein Borussia 09 e. V. Dortmund, commonly known as Borussia Dortmund (German pronunciation: [boˈʁʊsi̯aː ˈdɔɐ̯tmʊnt] (listen)), BVB (pronounced [beːfaʊ̯ˈbeː] (listen)), or simply Dortmund (pronounced [ˈdɔʁtmʊnt] (listen)), is a German professional sports club based in Dortmund, North Rhine-Westphalia.]
sim [(array([[0.52675045]], dtype=float32), Dortmund), (array([[0.52675045]], dtype=float32), Dortmund), (array([[0.10776877]], dtype=float32), ˈdɔʁtmʊnt)]

 selected object (array([[0.52675045]], dtype=float32), Dortmund) 

result  {'score': 0.3734171688556671, 'token': 5429, 'token_str': ' FC', 'sequence': 'Ballspielverein Borussia 09 e. V. Dortmund, commonly known as Borussia Dortmund (German pronunciation: [boˈʁʊsi̯aː ˈdɔɐ̯tmʊnt] (listen)), BVB (pronounced [beːfaʊ̯ˈbeː] (listen)), or simply Dortmund (pronounced [ˈdɔʁtmʊnt] (listen)), is a German professional sports club based in Dortmu